In [ ]:
# Imports
import pandas as pd
import numpy as np
import math
import regex as re
# for nlp tags
import spacy

#prevents warning
import warnings
warnings.filterwarnings('ignore')

# my functions
import sys
sys.path.insert(1,'PATH TO FILE')
import pscCatTypeComp as my
sys.path.remove('PATH TO FILE')

nlp = spacy.load("en_core_web_trf")

### For use in Nationality Extraction

In [ ]:
# loading nationality data so we can match different ways of stating same nationality
df_off = pd.read_csv("PATH TO FILE", encoding = 'iso-8859-1')

#NOC smart survey updated 1a dec

nationalities = list(df_off.OFFICER_NATIONALITY.apply(lambda x: x.lower() if type(x) == str else '').unique())
nationalities.remove('')
nationalities = [nat for nat in nationalities if ',' not in nat]


df_nat_and_adj = pd.read_csv("PATH TO FILE")

# making all lower case
for col in df_nat_and_adj.columns:
    df_nat_and_adj[col] = df_nat_and_adj[col].apply(lambda x: x.lower())
    
nationalities = nationalities + [nat for nat in df_nat_and_adj.country.values if nat not in nationalities] + [nat for nat in df_nat_and_adj.adjectivals.values if nat not in nationalities]    
nationalities = list(set(nationalities))

# Preprocessing

In [ ]:
def get_details(x):
    # initial splitting depending on whats contained in discrepancy report
    none_split = False # for checking if no splits were done
    
    if ',discrepancy - ' in x:
        first_split = x.split(',discrepancy - ')

    elif ',discrepancies - ' in x:
        first_split = x.split(',discrepancies - ') 
    
    elif ' discrepancy - ' in x:
        first_split = x.split(' discrepancy - ')
    
    else:
        none_split = True
    
    details = first_split[0] if none_split == False else x
    
    return details

def get_extract(x):
    # initial splitting depending on whats contained in discrepancy report
    none_split = False
    
    if ',discrepancy - ' in x:
        first_split = x.split(',discrepancy - ')

    elif ',discrepancies - ' in x:
        first_split = x.split(',discrepancies - ') 
    
    elif ' discrepancy - ' in x:
        first_split = x.split(' discrepancy - ')
    
    else:
        none_split = True
    
    extract = first_split[-1] if none_split == False else np.nan
    
    return extract

def get_extract_preserved(x):
    # initial splitting depending on whats contained in discrepancy report
    none_split = False
    
    if ',Discrepancy - ' in x:
        first_split = x.split(',Discrepancy - ')

    elif ',Discrepancies - ' in x:
        first_split = x.split(',Discrepancies - ') 
    
    elif ' discrepancy - ' in x:
        first_split = x.split(' discrepancy - ')
    
    else:
        none_split = True
    
    extract = first_split[-1] if none_split == False else np.nan
    
    return extract
def get_discrepancy_option_all(x):
    if type(x) == str:
        tags = ['legal form','governing law','place registered','nature of control','forename','surname','company number',
                    'nationality','month of birth','year of birth','place of residence','country of residence','company name',
               'correspondence address','incorporated in']

        found_tags = [tag for tag in tags if tag in x] # to append found tags to

        if 'month of birth' in found_tags or 'year of birth' in found_tags:
            if 'month of birth' in found_tags:
                found_tags.remove('month of birth')
            if 'year of birth' in found_tags:
                found_tags.remove('year of birth')
            found_tags.append('date of birth')
        
        if 'forename' in found_tags or 'surname' in found_tags:
            if 'forename' in found_tags:
                found_tags.remove('forename')
            if 'surname' in found_tags:
                found_tags.remove('surname')
            found_tags.append('name')
        
        if len(found_tags) == 0:
            # check for missed items in regards to nature of control
            noc_tags = ['shareholder','ownership','voting or control','voting rights','significant control','voting control',
                       '%','shareholding','shares','limited by guarantee','percent','per cent','owned','owner']
            noc_test = [True if tag in x else False for tag in noc_tags]
            
            if any(noc_test):
                found_tags = ['nature of control']
            elif 'inactive psc' in x or 'unauthorised psc' in x or 'duplicate psc' in x:
                found_tags = ['psc remove']
                
            elif 'address line' in x:
                found_tags = ['Address']
                
            elif x.replace(' ','').replace(',please provide details of "other" discrepancy - n/a','') == 'incorrectpscdetails':
                found_tags = ['incorrect psc details']
                
            elif x.replace(' ','').replace(',please provide details of "other" discrepancy - n/a','') == 'incorrectcompanydetails':
                found_tags = ['incorrect company details']
                
            elif 'missing' in x:
                found_tags = ['psc missing']
            
            elif any([True if item == x.replace(',','').replace('/','').replace('.','').replace(' ','') else False for item in ['na','','xxx','none']]):
                found_tags = ['other reason']
                
            else:
                found_tags = ['other reason']

    else:
        found_tags = np.nan
        
    return ', '.join(found_tags) if type(found_tags) == list else found_tags

def preprocess_smart_survey(df,df_act):
    
    # first need some cleaning on strings
    #df['Organization']=df['SURNAME']
    #df.drop(['SURNAME'], axis=1, inplace=True)

    forenames = ['first name(s)','first names(s)']
    
    df['Contact_CLOB_copy'] = df.Contact_CLOB.apply(lambda x: x.lower().replace(forenames[0],'forename').replace(forenames[1],'forename').replace('year of bith','year of birth'))
    
    # getting report information and discrepancy details
    df['information_provided'] = df['Contact_CLOB_copy'].apply(get_details)
    df['discrepancy_extract'] = df['Contact_CLOB_copy'].apply(get_extract)
    df['discrepancy_extract_cap_preserve'] = df.Contact_CLOB.apply(get_extract_preserved)
    
    # getting discrepancy_options_all by searching extract
    df['discrepancy_option_all'] = df['discrepancy_extract'].apply(get_discrepancy_option_all)
    
    return df

def merge_names_nat(arr):
    to_combine = []
    for item in arr:
        if type(item) != str:
            continue
        else:
            to_combine.append(item.lower())
            
    return ' '.join(to_combine)

def preprocess_officers(df):
    #df = df[~df.RESIGNATION_DATE.isna()]
    #df = df.drop_duplicates(['OFFICER_ID','NATURE_OF_CONTROL_TYPE_ID'])
    
    df['LB'] = df.DESCRIPTION.apply(lambda x: min([9999] + [int(item) for item in x.replace('%','').split(' ') if item.isdigit() == True]))
    df['UB'] = df.DESCRIPTION.apply(lambda x: max([0] + [int(item) for item in x.replace('%','').split(' ') if item.isdigit() == True]))
    df.loc[df.LB == 9999,'LB'] = np.nan
    df.loc[df.UB == 0,'UB'] = np.nan
    df.loc[(df.LB == 75) & (df.UB == 75),'UB'] = 100
    df['TYPE'] = df.DESCRIPTION.apply(lambda x: 'NOC' if 'shares' in x else 'VR' if 'voting' in x else 'SIC')
    df['AS'] = df.DESCRIPTION.apply(lambda x: 'trust' if 'trust' in x else 'person' if 'person' in x else 'firm')
    
    df['merged_name'] =  [merge_names_nat(x).replace('  ',' ') for x in df[['OFFICER_FORENAME_1','OFFICER_FORENAME_2','OFFICER_SURNAME']].values]
    
    return df

def preprocess_other_tables(df_off,df_act):
    # action code table first
    df_act['ACTION_CODE_EFF_DATE'] = pd.to_datetime(df_act.ACTION_CODE_EFF_DATE)
    df_act = df_act.sort_values(by = 'ACTION_CODE_EFF_DATE',ascending = False)
    
    # officers table
    df_off = preprocess_officers(df_off)
    
    return df_off,df_act

def given_name_extract(x):
    if type(x) != str:
        return np.nan
    
    if 'forename -' in x: # if forename found
        
        first_split = x.split('forename -')[1]
        first_name = first_split.split(',')[0]
        
        if 'surname -' in x: # check for surname
            second_split = x.split('surname -')[1]
            second_name = second_split.split(' ,')[0]
            
        else:
            second_name = ''
        
        full_name = ' | '.join([str(first_name),str(second_name)]) # join names
    
    elif 'surname -' in x: # if no forename but surname found
        full_name = x.split('surname -')[1].split(' ,')[0] # as we know 
        
    elif 'name -' in x: # if none of above but name given
        split = x.split('name -')[1]
        full_name = split.split(' ,')[0]
    
    else:
        return np.nan
    
    # if we haven't exited via the else then we need a bit of cleaning
    full_name.replace('?',"'") # changes i.e O?Brien to O'Brien and John   Doe to John Doe
    
    while '  ' in full_name:
        full_name = full_name.replace('  ',' ')
    
    while full_name[0] == ' ':
        full_name = full_name[1:]
    while full_name[0] == ' ':
        full_name = full_name[:-1]
        
    return full_name

def given_name_extract(x):
    if type(x) != str:
        return np.nan
    
    if 'forename -' in x: # if forename found
        
        first_split = x.split('forename -')[1]
        first_name = first_split.split(',')[0]
        
        if 'surname -' in x: # check for surname
            second_split = x.split('surname -')[1]
            second_name = second_split.split(' ,')[0]
            
        else:
            second_name = ''
        
        full_name = ' | '.join([str(first_name),str(second_name)]) # join names
    
    elif 'surname -' in x: # if no forename but surname found
        full_name = x.split('surname -')[1].split(' ,')[0] # as we know 
        
    elif 'name -' in x: # if none of above but name given
        split = x.split('name -')[1]
        full_name = split.split(' ,')[0]
    
    else:
        return np.nan
    
    # if we haven't exited via the else then we need a bit of cleaning
    full_name.replace('?',"'") # changes i.e O?Brien to O'Brien and John   Doe to John Doe
    
    while '  ' in full_name:
        full_name = full_name.replace('  ',' ')
    
    while full_name[0] == ' ':
        full_name = full_name[1:]
    while full_name[0] == ' ':
        full_name = full_name[:-1]
        
    return full_name

def get_recent_act(cont_id,df_act):
    our_df = df_act[df_act.CORPORATE_BODY_ID == cont_id]
    
    if our_df.shape[0] == 0:
        return 'NO CURRENT ACTION'
    
    date_to_check = our_df.ACTION_CODE_EFF_DATE.values[0]
    
    codes = our_df[our_df.ACTION_CODE_EFF_DATE == date_to_check].ACTION_CODE_DESC.values
    
    return our_df.ACTION_CODE_TYPE_ID.values[0] if 'NO CURRENT ACTION' not in codes else 0    

def get_recent_act_desc(cont_id,df_act):
    our_df = df_act[df_act.CORPORATE_BODY_ID == cont_id]
    
    if our_df.shape[0] == 0:
        return 'NO CURRENT ACTION'
    
    date_to_check = our_df.ACTION_CODE_EFF_DATE.values[0]
    
    codes = our_df[our_df.ACTION_CODE_EFF_DATE == date_to_check].ACTION_CODE_DESC.values
    
    return our_df.ACTION_CODE_DESC.values[0] if 'NO CURRENT ACTION' not in codes else 0      

def load_preprocess_data(path,
        survey_file = 'smart survey 7Dec.csv',action_file = 'action code smart survey 6 Dec.csv',  officer_file = 'NOC smart survey 6Dec dates.csv'):
    
    # loading data
    df = pd.read_csv(path+survey_file,sep=';', encoding = 'iso-8859-1')
    df_off = pd.read_csv(path+officer_file,encoding = 'iso-8859-1')
    df_act = pd.read_csv(path+action_file,encoding = 'iso-8859-1')
    
    # preprocessing
    df = preprocess_smart_survey(df,df_act)
    df['PSC_NAME'] = df.information_provided.apply(given_name_extract).apply(lambda x: x.replace(' | ',' ') if type(x) == str else x)
    df.loc[df.PSC_NAME.isna(),'PSC_NAME'] = df.discrepancy_extract.apply(given_name_extract).apply(lambda x: x.replace(' | ',' ') if type(x) == str else x)
    
    df_off,df_act = preprocess_other_tables(df_off,df_act)
    
    print('Before Action Code removals : ',df.shape[0])
    
    action_codes_avoid = [
    41,4100,41,43,4300,44,4400,50,5000,
    70,7001,7002,71,7101,73,7301,7303,
    7304,7307,7308,74,7401,7402,7403,
    7404,76,7601,7701,90,9000,9100
    ]
    
    df['act_code'] = df.CORPORATE_BODY_ID.apply(lambda x: get_recent_act(x,df_act))
    df['act_code_desc'] = df.CORPORATE_BODY_ID.apply(lambda x: get_recent_act_desc(x,df_act))
    
#    psc_ids_preserve = [91287,88974,93819]
#    preserve_inds = [df[df.PSC_DISCREPANCY_ID == psc_id].index[0] for psc_id in psc_ids_preserve]
#    df.loc[preserve_inds,'act_code'] = 0
    
    act_bool = [False if x in action_codes_avoid else True for x in df.act_code.values]
    act_removed_bools = [True if x in action_codes_avoid else False for x in df.act_code.values]
    df_return = df.loc[act_bool,:]
    df_removed = df.loc[act_removed_bools,:]
    print('After : ',df_return.shape[0])
    
    return df_return,df_off,df_act,df_removed

# Extractions

In [ ]:
def get_relevant_officer(off_df,psc_name,default_prio = 'High'):
    # function to be used on df_off once reduced by contact id.
    # will return string of officer name OR appropriate priority_explained, priority o/p's
    
    if type(psc_name) != str:
        return 'PSC name extraction failure',default_prio
    
    exact_check = off_df[off_df.merged_name == psc_name]
    
    if exact_check.shape[0]>0:
#        print("\n\n=======> FOUND MATCH <=======")
        officer_name = psc_name # single value of matched officer
    else:
        # Doing more thorough matching methods
        unq_officers = off_df.merged_name.unique()
        no_offs = len(unq_officers) # for below and loop
        off_scores = np.zeros(no_offs) # for scores to be appended to
        
        # first check with provided name
        for part in psc_name.split(' '): # iterates through part of name from above current item
            for i in range(no_offs): # checks then against all officers in CHIPS

                to_compare = unq_officers[i] # gets current officer

                for off_part in to_compare.split(' '): # checks every part of officer name against our PSC
                    if off_part == part: # if matches increase by score 1
                        off_scores[i] +=1
        print(unq_officers)
        print(off_scores)
        sh_choice = np.where(off_scores == np.amax(off_scores)) # gets indices of best matches
        
        if len(sh_choice[0]) > 1:
#            print("\n\n =========> MULTI MATCHED <=========")
#            print('\n===> OFFICER NAME(S) : ',unq_officers[sh_choice[0]])
            return 'Multi-matched','High' # if many matches need manual look so high
            
        elif sum(off_scores) == 0:
            return 'No officer matches found','High' # returning no matches, high
        
        else:
#            print("\n\n=======> FOUND MATCH v2 <=======")
            officer_name = unq_officers[sh_choice[0][0]] #  single value of matched officer
            
    return officer_name

def mon_extract(x):
    if 'month of birth -' in x:
        first_split = x.split('month of birth -')[1]
        month = first_split.split(' ,')[0]
        
        month = str(month.replace(' ',''))
        
        return month
    
    else:
        return np.nan
    
def year_extract(x):
    if 'year of birth -' in x:
        second_split = x.split('year of birth -')[1]
        year = second_split.split(' ,')[0]
        
        year = str(year.replace(' ',''))
        
        return str(year)
    else:
        return np.nan
    
def dob_extract(x):
    mon = mon_extract(x)
    yr = year_extract(x)
    
    mon = str(mon) if 'nan' not in str(mon) else ''
    yr = str(yr) if 'nan' not in str(yr) else ''
        
    mon = mon if len(mon) == 2 else '0'+mon if mon != '' else mon
    return mon + '/' +yr

def nationality_extract(x):
    if 'nationality - ' in x:
        split = x.split('nationality - ')[1]
        nat = split.split(' ,')[0]
        return nat
    else:
        found_nats = []
        for nat in nationalities:
            if nat in x:
                found_nats.append(nat)
            
        found_nats = list(set(found_nats))
        new_nats = []

        for nat in found_nats:
            if nat in df_nat_and_adj.country.values:
                new_nat = df_nat_and_adj[df_nat_and_adj.country == nat].adjectivals.values[0]
            elif nat in df_nat_and_adj.adjectivals.values:
                new_nat = df_nat_and_adj[df_nat_and_adj.adjectivals == nat].country.values[0]
                new_nat = df_nat_and_adj[df_nat_and_adj.country == new_nat].adjectivals.values[0]
            else:
                new_nat = nat
                
            if new_nat not in new_nats and new_nat != 'other':
                new_nats.append(new_nat)
        
        if len(new_nats) == 0:
            return np.nan
        else:
            return ','.join(new_nats)

def company_number_extract(x):
    if 'company number - ' in x:
        split = x.split('company number - ')
        comp_num = split[1].split(' ,')[0]
        return comp_num
    else:
        return np.nan
    
def company_name_extract(x):
    if 'company name - ' in x:
        split = x.split('company name - ')[1]
        comp_name = split.split(' ,')[0]
        return comp_name
    else:
        return np.nan
    
def noc_extract(psc_id):
    ind = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    psc_name = df.loc[ind,'PSC_NAME']
    cont_id = df.loc[ind,'CORPORATE_BODY_ID']
    off_df = df_off[df_off.CORPORATE_BODY_ID == cont_id]
    
#    print("\n ===> INSIDE FUNC. <===")
#    print("===> PSC NAME : ",psc_name)
    
    if off_df.shape[0] == 0:
        return 'No officers found'
        
    officer = get_relevant_officer(off_df,psc_name) # finds our officer
    
#    print("===> OFFICER : ",officer)
    
    
    rel_df = off_df[off_df.merged_name == officer].drop_duplicates(['NATURE_OF_CONTROL_TYPE_ID','OFFICER_ID'])
    
    if len(rel_df.OFFICER_ID.unique())>1:
        print("\n\n======================================================================================================================================================================================================================")
        print("\n => PSC ID : ",psc_id)
        print("\n => Contact_CLOB : ",df.loc[ind,'Contact_CLOB'])
        print("\n => OFFICERS : \n")
        print(rel_df)
        return 'Too many officers with name match'
    
    
    #print("===> OFFICER CONTROLS : \n",off_df,"\n")
    
    controls = {'NOC':None,'VR':None,'SIC':None}
    for i in rel_df.index:
        type_control = rel_df.loc[i,'TYPE']
        
        if type_control != 'SIC': # if not SIC then get LB and UB
            LB = rel_df.loc[i,'LB']
            UB = rel_df.loc[i,'UB']
            
            if controls[type_control] != None : # if already listed then tag multiple
                controls[type_control] = 'Multiple'
            else:
                controls[type_control] = str(LB)+' to '+str(UB) # if not then store as "LB,UB"

        elif controls[type_control] != None: # if contro SIC but alreayd listed
            controls[type_control] = 'Multiple' # flag as multi if above is true
        else:
            controls[type_control] = type_control # if no SIC then store SIC
    
    controls_to_return = ' | '.join([x + ': ' + controls[x]  for x in controls.keys() if controls[x] != None])
    controls_to_return = controls_to_return.replace('NOC','SH').replace('SIC','RTA') 
    
    if controls_to_return == '':
        controls_to_return = 'No controls found'
    return controls_to_return

def run_extractions(df):
    func_for_type = {'name':given_name_extract,'date of birth':dob_extract,'nationality':nationality_extract,
                     'company number':company_number_extract,'company name':company_name_extract}
    
    # extracting on both information and discrepancy options single discrepancies
    df['relevant_information'] = np.nan
    df['discrepancy_information'] = np.nan
    
    # iterating through keys above
    for discrep_option in func_for_type.keys():
        print("\n\n===> CURRENT KEY : ",discrep_option,"\nSize : ",df[df.discrepancy_option_all == discrep_option].shape[0])
        df.loc[df.discrepancy_option_all == discrep_option,'relevant_information'] = df.loc[df.discrepancy_option_all == discrep_option,'information_provided'].apply(func_for_type[discrep_option])
        df.loc[df.discrepancy_option_all == discrep_option,'discrepancy_information'] = df.loc[df.discrepancy_option_all == discrep_option,'discrepancy_extract'].apply(func_for_type[discrep_option])
    
    
    # psc_id types
    psc_id_types = {'nature of control':noc_extract}
    for discrep_option in psc_id_types.keys():
        print("\n\n===> CURRENT KEY : ",discrep_option,"\nSize : ",df[df.discrepancy_option_all == discrep_option].shape[0])
        df.loc[df.discrepancy_option_all == discrep_option,'relevant_information'] = df.loc[df.discrepancy_option_all == discrep_option,'PSC_DISCREPANCY_ID'].apply(psc_id_types[discrep_option])
    
    print("\n ===> DONE <===")
    return df



# Comparisons

In [ ]:
def name_compare(psc_id):
    # getting relevant information
    ind = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    provided = df.loc[ind,'relevant_information'] # NAME CURRENTLY IS
    discrep = df.loc[ind,'discrepancy_information'] # NAME SHOULD BE
    cont_id = df.loc[ind,'CORPORATE_BODY_ID'] # for getting officer information
    off_df = df_off.loc[df_off.CORPORATE_BODY_ID == cont_id] # selection of df_off (officer details) relevant to query
    discrep_details = df.loc[ind,'discrepancy_extract']
    
    # first check extraction worked...
    if type(discrep) != str:
        return ['Extraction error - Name','High',None]
    
    # checking for officer : different to other sections as more complex approach used with NAME discrepancy option
    exact_check = off_df[off_df.merged_name == provided]
    
    if exact_check.shape[0]>0:
#        print("\n\n=======> FOUND MATCH <=======")
        officer_name = provided # returns list with single value of matched officer
    else:
        # Doing more thorough matching methods
        unq_officers = off_df.merged_name.unique()
        no_offs = len(unq_officers) # for below and loop
        off_scores = np.zeros(no_offs) # for scores to be appended to
        
        # first check with provided name
        for item in [provided,discrep.replace(' | ','')]: # checks provided then discrepancy
            if type(item) != str:
                continue
            for part in item.split(' '): # iterates through part of name from above current item
                for i in range(no_offs): # checks then against all officers in CHIPS
        
                    to_compare = unq_officers[i] # gets current officer
            
                    for off_part in to_compare.split(' '): # checks every part of officer name against our PSC
                        if off_part == part: # if matches increase by score 1
                            off_scores[i] +=1
                
        if len(off_scores) == 0:
            return ['Matching error - Name','High',None]
        
        sh_choice = np.where(off_scores == np.amax(off_scores)) # gets indices of best matches
        
        if len(sh_choice[0]) > 1:
#            print("\n\n =========> MULTI MATCHED <=========")
#            print('\n===> OFFICER NAME(S) : ',unq_officers[sh_choice[0]])
            return ['Multi-matched - Name','High','/'.join(unq_officers[sh_choice[0]])] # if many matches need manual look so high
            
        elif sum(off_scores) == 0:
            return ['No officer matches found - Name','High',None] # returning no matches, high
        
        else:
#            print("\n\n=======> FOUND MATCH v2 <=======")
            officer_name = unq_officers[sh_choice[0][0]] # returns list with single value of matched officer
            
    # Comparing Match to Update, if exact match with discrep then we disregard report else we report with priority        
    discrep_part = discrep.replace(' -','-').replace('- ','-')# splits into parts and cleans double barrels
    discrep_part = discrep_part.split(' | ') if ' | ' in discrep_part else discrep_part.split(' ')
    
    discrep_surname = discrep_part[-1] # surname of discrepancy detail name
    discrep_forename = ' '.join(discrep_part[:-1]) # forename of discrepancy detail name
    
#    print("\n===> DISCREP SURNAME(S) : ",discrep_surname)
#    print("\n===>         FORENAMES(S) : ",discrep_forename)
    
    exact_df = off_df[off_df.merged_name == officer_name]
    off_surs = exact_df.OFFICER_SURNAME.apply(lambda x:x.lower()).unique()[0] # gets all officer surnames
    exact_df['fores_merged'] = [my.merge_names_nat(x).replace('  ',' ') for x in exact_df[['OFFICER_FORENAME_1','OFFICER_FORENAME_2']].values] # get all officer forenames 
    off_fores = exact_df.fores_merged.unique()[0]
    
#    print('\n===> OFFICER SURNAME(S) : ',off_surs)
#    print('\n===>         FORENAME(S) : ',off_fores)
    
    if discrep_surname != off_surs: # checks for exact surname matches
        marriage_tags = ['maiden','marriage','married']
        return_prio = 'Medium' if my.how_different(discrep_surname,off_surs) <= 2 else 'High'
        
        if any([True if tag in discrep_details else False for tag in marriage_tags]):
            return ['Marriage difference','Low',officer_name]
        else:
            return ['Surname difference',return_prio,officer_name] 
    
    elif discrep_forename != off_fores:
        return ['Forename difference','None',officer_name]
    else:
        return ['All matched - Name','None',officer_name]
    
    return 

def nationality_compare(psc_id):
    # getting relevant information
    ind = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    provided = df.loc[ind,'relevant_information'] # NAT CURRENTLY IS
    discrep = df.loc[ind,'discrepancy_information'] # NAT SHOULD BE
    cont_id = df.loc[ind,'CORPORATE_BODY_ID'] # for getting officer information
    off_df = df_off.loc[df_off.CORPORATE_BODY_ID == cont_id] # selection of df_off (officer details) relevant to query
    discrep_details = df.loc[ind,'discrepancy_extract']
    psc_name = df.loc[ind,'PSC_NAME']
#    print("\n --------------------------------- INSIDE FUNCTION ---------------------------------")
#    print("\n ===> PSC Name : ",psc_name)
    
    # first check extraction worked...
    if type(discrep) != str: # nothing extraction
        return ['Extraction error - Nationality','Low',None] # issue return Low (Nationality priority == Low at most)
            
    if off_df.shape[0] == 0:
        return ['No officer match - Nationality','Low',None]
    
    officer_name = get_relevant_officer(off_df,psc_name) # gets officer name
    
#    print("\n ===> OFFICER NAME : ",officer_name)
#    print("\n ===> OFFICERS : ",off_df.merged_name.unique())
    
    if type(officer_name) != str: # if officer not found or multiple tuple is returned with correct o/p
        return ['No officer match - Nationality','Low',None]
    
    officer_nats = list(off_df.loc[off_df.merged_name == officer_name,'OFFICER_NATIONALITY'].apply(lambda x: x.lower()).unique())
    
    # for when two nationalities stored in CHIPS
    extract_off_nats = []
    for nat in officer_nats:
        if ',' in nat:
            nat_split = nat.split(',')
        elif '/' in nat:
            nat_split = nat.split('/')
        elif '.' in nat:
            nat_split = nat.split('.')
        elif '-' in nat:
            nat_split = nat.split('-')
        else:
            nat_split = [nat]
        
        for new_nat in nat_split:
            if new_nat not in extract_off_nats:
                extract_off_nats.append(new_nat)
    
    officer_nats = extract_off_nats
            
    # changing all officer nationalities to most common adjectival for country
    reduced_items = []
    for norp in officer_nats:
        norp = norp.lower()
        if norp in df_nat_and_adj.country.values:
            new_name = df_nat_and_adj[df_nat_and_adj.country == norp].adjectivals.values[0]
            reduced_items.append(new_name)
            
        elif norp in df_nat_and_adj.adjectivals.values:
            our_country = df_nat_and_adj[df_nat_and_adj.adjectivals == norp].country.values[0]
            new_name = df_nat_and_adj[df_nat_and_adj.country == our_country].adjectivals.values[0]
            reduced_items.append(new_name)
            
        else:
            reduced_items.append(norp)
    
    reduced_items = list(set(reduced_items))
    
    # changing discrepency nationality to be most common adjectival
    if type(discrep) != str:
        return ['Extraction Error - Nationality','Low',reduced_items]
    
    if ',' in discrep:
        discrep_nats = discrep.split(',')
    elif '/' in discrep:
        discrep_nats = discrep.split('/')
    elif '.' in discrep:
        discrep_nats = discrep.split('.')
    elif '-' in discrep:
        discrep_nats = discrep.split('-')
    else:
        discrep_nats = discrep.split(' ')
        
    new_discrep = []
    for norp in discrep_nats:
        norp = norp.lower()
        if norp in df_nat_and_adj.country.values:
            new_name = df_nat_and_adj[df_nat_and_adj.country == norp].adjectivals.values[0]
            new_discrep.append(new_name)
            
        elif norp in df_nat_and_adj.adjectivals.values:
            our_country = df_nat_and_adj[df_nat_and_adj.adjectivals == norp].country.values[0]
            new_name = df_nat_and_adj[df_nat_and_adj.country == our_country].adjectivals.values[0]
            new_discrep.append(new_name)
            
        else:
            new_discrep.append(norp)
    
    new_discrep = list(set(new_discrep))
    
#    print("\n ===> NEW DISCREP NATS : ",new_discrep)
#    print("\n ===> NEW OFFICER NATS : ",reduced_items)
    
    # now comparing...
    nat_test = [True if nat not in reduced_items else False for nat in new_discrep] # returns True if discrepancy nationality not in officer nationalities
    
    if any(nat_test):
        return ['Nationality difference','Low','/'.join(officer_nats)]
    else:
        return ['All matched - Nationality','None','/'.join(officer_nats)]
        

def company_name_compare(psc_id):
    # getting relevant information
    ind = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    provided = df.loc[ind,'relevant_information'] # NAME CURRENTLY IS
    discrep = df.loc[ind,'discrepancy_information'] # NAME SHOULD BE
    
#    print('\nPROVIDED : ',provided)
#    print('\nSHOULD BE : ',discrep)
    
    if provided != discrep:
        return ['Company name difference','High',provided]
    else:
        return ['All matched - Company name','None',provided]

def company_number_compare(psc_id):
    # getting relevant information
    ind = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    provided = df.loc[ind,'relevant_information'] # NAME CURRENTLY IS
    discrep = df.loc[ind,'discrepancy_information'] # NAME SHOULD BE
    
#    print('\nPROVIDED : ',provided)
#    print('\nSHOULD BE : ',discrep)
    
    if provided != discrep:
        return ['Company number difference','High',provided]
    else:
        return ['All matched - Company number','None',provided]
    
def dob_compare(psc_id):
    # getting relevant information
    ind = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    provided = df.loc[ind,'relevant_information'] # NAME CURRENTLY IS
    discrep = df.loc[ind,'discrepancy_information'] # NAME SHOULD BE
    cont_id = df.loc[ind,'CORPORATE_BODY_ID'] # for getting officer information
    off_df = df_off.loc[df_off.CORPORATE_BODY_ID == cont_id] # selection of df_off (officer details) relevant to query
    discrep_details = df.loc[ind,'discrepancy_extract']
    psc_name = df.loc[ind,'PSC_NAME']
    
    # first check extraction worked & details found...
    if type(discrep) != str: # nothing extraction 
        return ['Extraction error - Date of birth','High',None] # issue return Low (Nationality priority == Low at most)
    
    if off_df.shape[0] == 0:
        return ['No officer information found - Date of birth','High',None]
    
    officer_name = get_relevant_officer(off_df,psc_name) # gets officer name
    
    #print("\n ===> OFFICER NAME : ",officer_name)
    #print("\n ===> OFFICERS : ",off_df.merged_name.unique())
    
    if type(officer_name) != str: # if officer not found or multiple tuple is returned with correct o/p
        return ['No officer match - Date of birth','High',None]
    
    off_df = off_df[off_df.merged_name == officer_name]
    test = off_df.OFFICER_DATE_OF_BIRTH.unique()
    
    off_df.OFFICER_DATE_OF_BIRTH = pd.to_datetime(off_df.OFFICER_DATE_OF_BIRTH)
    officer_dobs = off_df.OFFICER_DATE_OF_BIRTH.unique()
    
    if len(officer_dobs) > 1:
        return ['Multi-D.o.B - Date of birth','High',None]
    
    # officer dob
    found_month = officer_dobs[0].astype('datetime64[M]').astype(int) % 12 + 1 # get officer month
    found_year = officer_dobs[0].astype('datetime64[Y]').astype(int) + 1970 # get officer year
    
    found_year = found_year if found_year < 2021 else found_year - 100 # is correction for numpy package error
    
    # correction dob
    extract_mon = discrep.split('/')[0]
    extract_year = discrep.split('/')[1]
    
    extract_year = int(extract_year) if extract_year.isdigit() else None
    extract_mon = int(extract_mon) if extract_mon.isdigit() else None # for when no month is passed
    
    #print('\n\n ===> FOUND MONTH : ',found_month)
    #print('\n ===> FOUND YEAR : ',found_year)
    
    #print('\n\n ===> EXTRACT MONTH : ',extract_mon)
    #print('\n ===> EXTRACT YEAR : ',extract_year)
    
    # CHIPS PART TO REPORT
    CHIPS_val = str(found_month)+'/'+str(found_year)
    
    # comparison
    if extract_mon == None: # if no month passed only check year
        if extract_year != found_year:
            return ['Difference found - Date of birth','High',CHIPS_val]
        else:
            return ['All matched - Date of birth','None',CHIPS_val]
        
    elif (extract_mon != found_month) or (extract_year != found_year):
        return ['Difference found - Date of birth','High',CHIPS_val]
    else:
        return ['All matched - Date of birth','None',CHIPS_val]
        
    return

def noc_compare(psc_id,trace = False):
    ind = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    provided = df.loc[ind,'relevant_information'] # Current controls from CHIPS but stored here due to order of extraction
    discrep_details = df.loc[ind,'discrepancy_extract']
    psc_name = df.loc[ind,'PSC_NAME']
    cont_id = df.loc[ind,'CORPORATE_BODY_ID']
    off_df = df_off[df_off.CORPORATE_BODY_ID == cont_id].drop_duplicates(['NATURE_OF_CONTROL_TYPE_ID','OFFICER_ID'])
    officers = off_df.merged_name.unique()
    tracer = '\n===> INSIDE FUNC. <===\n'
    
    # removal of address line parts
    while 'address line' in discrep_details:
        discrep_split = discrep_details.split('address line')
        part_a = discrep_split[0]
        part_b = ','.join('address line'.join(discrep_split[1:]).split(',')[1:])
        discrep_details = part_a+part_b
        
    if 'postcode ' in discrep_details:
        discrep_split = discrep_details.split('postcode')
        part_a = discrep_split[0]
        part_b = ''.join(discrep_split[1].split(',')[1:])
        discrep_details = part_a + part_b
    
    tracer += '\n==> Discrep Add Rem : '+discrep_details
    if type(psc_name) != str:
        if trace == True:
            print(tracer)
        return ['No officer provided - NOC','High',provided]
    
    title_removals = ['mr','mrs','ms','mx','miss']
    for title in title_removals:
        psc_name = psc_name.replace(title+' ','')
    
    discrep_pres = df.loc[ind,'discrepancy_extract_cap_preserve']
    pos_tags = my.get_pos_tags(discrep_pres,nlp)
 
    tracer += '\n\n===> OFFICERS : '+str(officers)
    off_fores = [off.split(' ')[0] for off in officers]
    off_in_text = [True if off.split(' ')[0] in discrep_details else False for off in officers]
    names_in_text = [off for off in officers if off.split(' ')[0] in discrep_details]
    
    if 'Too many officers with name match' == provided:
        tag = 'High'
        text = 'Nature of Control in text'
        if 'voting' in discrep_details and 'shares' not in discrep_details and 'shareholding' not in discrep_details and 'financial ownership' not in discrep_details and 'capital' not in discrep_details:
            tag = 'Low'
            text = 'Missing VR - NOC'
        return [text,tag,off_df[off_df.merged_name == psc_name]]
    
    if psc_name.split(' ')[0] not in off_fores or 'add psc' in discrep_details:
        if trace == True:
            print(tracer)
        return ['Add PSC - NOC','High',psc_name]
    
    if sum(off_in_text) > 1:
        if trace == True:
            print(tracer)
        return ['Multiple persons - NOC','High',names_in_text]
    
    
    if type(provided) == float:
        return ['No controls found - NOC','High',names_in_text]
    
    if 'Multiple' in provided: # if multi controls found in officer info then flag
        if trace == True:
            print(tracer)
        tag = 'High'
        text = 'Nature of Control in text'
        if 'voting' in discrep_details and 'shares' not in discrep_details and 'shareholding' not in discrep_details and 'financial ownership' not in discrep_details and 'capital' not in discrep_details:
            tag = 'Low'
            text = 'Missing VR - NOC'
        return [text,tag,provided]
    else:
        current_controls = {'SH':None,'VR':None,'RTA':None}
        
        for control_type in current_controls:
            if control_type in provided:
                current_controls[control_type] = provided.split(control_type+': ')[-1].split(' | ')[0].split(' to ')
                current_controls[control_type] = [float(x) if x.split('.0')[0].isdigit() else x for x in current_controls[control_type]]
                
    tracer += "\n===> CURRENT CONTROLS : " + str(current_controls) 
    # for reported controls to be appended to
    reported_controls = {'SH':[None],'VR':[None],'RTA':[None]}
    
    
    ### EXTRACTION FROM FREE TEXT ###
    
    # for bot using aggregated key word
    if 'aggregated percentage' in discrep_details: # we note many with a same format with the word aggregated which does not appear in many others
        relevant_part = discrep_details.split(':')[-1]
        
        if 'capital' in relevant_part:
            reported_controls['SH'] = [relevant_part.split('capital')[-1][1:].split('%')[0]] # gets only what is stated for capital/ownershiop
        
        if 'voting or control' in relevant_part:
            reported_controls['VR'] = [relevant_part.split('voting or control')[-1][1:].split('%')[0]] # same as above but for voting rights
        
            
    # secondary bot check
    elif 'how much of your business do they financially own ' in discrep_details or 'voting ownership ' in discrep_details or 'financial ownership ' in discrep_details:
        if 'how much of your business do they financially own' in discrep_details: # ownership reporting
            to_append = discrep_details.split('how much of your business do they financially own ')[-1].split('%')[0]
            reported_controls['SH'] = [to_append] if 'none' not in to_append.lower() else [0] # check for when none is reported
            
        elif 'financial ownership ' in discrep_details:
            to_append = discrep_details.split('financial ownership ')[-1].split('%')[0]
            reported_controls['SH'] = [to_append] if 'none' not in to_append.lower() else [0] # check for when none is reported
            
        if 'voting ownership ' in discrep_details: # voting reporting
            to_append = discrep_details.split('voting ownership ')[-1].split('%')[0]
            reported_controls['VR'] = [to_append] if 'none' not in to_append.lower() else [0]
    
    elif 'voting' not in discrep_details: # when voting isn't mentioned we can assume all to be about shares
        text_split = discrep_details.replace('-',' ').replace('%','').replace("\r",' ').replace('/',' ').split(' ')
        digits = [x for x in text_split if x.replace('.','').isdigit()]
        reported_controls['SH'] = digits if len(digits) > 0 else [None]
    
    
    # If not picked up by simpler methods then...
    else:

        discrep_split = discrep_details.replace('\n',' ').replace('\r',' ').split(' ')
        while '' in discrep_split:
            discrep_split.remove('')
            
        new_vals = {'SH':[],'VR':[]}
        
        curr_items = []
        
        i = 0
        while i < len(discrep_split):
            word = discrep_split[i]
            
            if word in ['shares','owns','holds']:
                curr_items.append('SH')
                i+=1
                continue
                
            if word in ['voting','vote']:
                curr_items.append('VR')
                i+=1
                continue
                
            if '%' in word or all([True if char.isdigit() == True or char =='.' else False for char in word ]):
                num = ''.join([char for char in word if char.isdigit()])
                num = float(num) if num != '' else 0
                if num >=10 and num <=100:
                    curr_items.append(num)
                
            i +=1
            
        tracer += "\n===> ITEMS : "+str(curr_items)
        
        key_ints = [25,50,75]
        i = 0
        append_to = []
        vals = []
        
        time_to_append = False
        tracer += "\nLEN CURR : " + str(len(curr_items))
        
        while i <= len(curr_items) :
            
            # check if ready to append
            if (time_to_append == True and len(append_to) > 0) or i == len(curr_items):
                tracer += "\n==> APPENDING : "+str([append_to,vals])
                if len(append_to) > 0:
                    for cat in append_to:
                        for val in vals:
                            reported_controls[cat].append(val)
                else:
                    for val in vals:
                        reported_controls['SH'].append(val)
                        
                time_to_append = False
                
                vals = []
                append_to = []
                
                if i == len(curr_items):
                    i = len(curr_items) + 1
                    
                continue
            
            item = curr_items[i]         
            
            if i != len(curr_items)-1:
                next_item = curr_items[i + 1]  
            else:
                next_item = None
            
            if item in ['SH','VR']: # if cat tag
                if len(append_to) > 0 and len(vals) > 0:
                    time_to_append = True
                    continue
                    
                if next_item in ['SH','VR'] and next_item != item:
                    append_to.append('VR')
                    append_to.append('SH')
                    i+=2
                    continue
                else:
                    append_to.append(item)
                    i += 1
                    continue
                
            elif next_item in key_ints and item in key_ints and next_item != item:
                vals.append(item)
                vals.append(next_item)
                i += 2
                time_to_append = True 
                continue
            else:
                vals.append(item)
                i+=1
                time_to_append = True
                continue
        
        for cat in ['SH','VR']:
            while None in reported_controls[cat]:
                reported_controls[cat].remove(None)
        
        
    # look for RTA
    if 'right to appoint' in discrep_details or 'influence or control' in discrep_details or 'significant control' in discrep_details:
        reported_controls['RTA'] = ['RTA']
    
    # Putting converting to int & floats
    for control_type in reported_controls.keys(): 
        controls = [None if x == None else x if type(x) != str else float(x) if x.split('.')[0].isdigit() else x for x in reported_controls[control_type]]
        controls = [x if type(x) != float else x if x > 10 and x <=100 else '' for x in controls]
        while '' in controls:
            controls.remove('')
        reported_controls[control_type] = controls
        
    tracer += "\n===> EXTRACT : " + str(reported_controls)
    
    
    ### COMPARISON TO CHIPS ### 95767 <-- to check
    test_for_no_info = all([x[0] == None for x in reported_controls])
    if test_for_no_info :
        if trace == True:
            print(tracer)
        return ['Extraction error - NOC','High',provided]
    
    
    
    reported_differences = {'SH':False,'VR':False,'RTA':False} # to be turned to true if mentioned
    tracer += '\n\n===> COMPARISONS :'
    for item in reported_controls.keys(): # iterating through types
        current = current_controls[item]
        reported = reported_controls[item]
        
        if len(reported) == 0:
            continue
        
        tracer+="\n===>CURRENT/VALUES : " + str([item,reported])
        
        if reported[0] == None and current == None: # If not relevant
            continue
        
        #checking for zeroes
        if reported[0] == 0 and current == None:
            continue
        elif reported[0] == 0 and current != None:
            reported_differences[item] = True
            continue
            
        if reported[0] != None and current == None: # if reported value but missing in CHIPS
            reported_differences[item] = True # report difference
            continue
            
        if len(reported) == 1 and type(reported[0]) == float and len(current) == 2: # if reported as 33.33% NOT 25% to 50%
            tracer += "\nCURRENTLY COMPARING : " + str(reported) + " with CHIPS: " + str(current)
            if reported[0] != 75:
                if reported[0] > current[0] and reported[0] <= current[1]: # does it sit within current bounds?
                    continue # if it does continue or ignor
            elif reported[0] == 75:
                if reported[0] == current[0]:
                    continue
            elif reported[0] <= 25:
                continue
            
            reported_differences[item] = True # report the difference
            
        if len(reported) > 1: # for when many items found
            tracer += "\n\n============MULTI============"
            tracer += "\n" + str(reported)
            # first we must group them
            groups = []
            i = 0
            while i < len(reported):
                
                if i+1 < len(reported):
                    if reported[i] == reported[i+1]-25:
                        groups.append([reported[i],reported[i+1]])
                        reported
                        i += 2
                        continue
                if reported[i] == 75:
                    groups.append([75])
                    i +=1
                    continue
                    
                groups.append([reported[i]])
                i+=1
                
            tracer += "\n===> GROUPS : "+str(groups)
            
            # now we go through groups and report differences
            for group in groups:
                if len(group) == 1:
                    if group[0] == 75 and current[0] != 75:
                        reported_differences[item] = True
                        continue
                    elif group[0] == 100 and current[1] == 100:
                        continue
                        
                    if group[0] > current[0] and group[0] <= current[1]:
                        continue
                    else:
                        reported_differences[item] = True
                elif len(group) == 2:
                    if group[0] != current[0] or group[1] != current[1]:
                        reported_differences[item] = True
        
    CHIPS_vals = [current_controls[cat] for cat in current_controls.keys()]
    our_vals = [list(set(reported_controls[cat])) for cat in reported_controls.keys()]
    diffs = [reported_differences[cat] for cat in reported_differences.keys()]
    
    for cat in reported_differences:
        reported_differences[cat] = list(set(reported_differences[cat])) if type(reported_differences[cat]) == list else reported_differences[cat]  
    
    convert_controls = lambda controls: ' | '.join([x + ': ' + str(controls[x]).replace('[','').replace(']','')  for x in controls.keys() if controls[x] != None]).replace(',',' to ')
    CHIPS_return = convert_controls(current_controls)
    report_return = convert_controls(reported_controls)
    diffs_return = convert_controls(reported_differences)
    
    op_final = '\n\n=============== OUTPUT ===============\n\nCHIPS Values : '+str(CHIPS_return) + '\n\nReported Controls : ' + str(report_return) + '\n\nDifferences : ' + str(diffs_return) 
    tracer += op_final

    if trace == True and 'aggregated' not in discrep_details:
        print(tracer)
    
    if sum(diffs) == 0:
        return ['All matched - NOC','None',report_return,CHIPS_return]
    elif diffs[1] == True and current_controls['VR'] == None and diffs[0] == False and diffs[2] == False:
        return ['Missing VR - NOC','Low',report_return,CHIPS_return]
    elif any(diffs):
        return ['Difference found - NOC','High',report_return,CHIPS_return]
    else:
        return ['Insufficient information - NOC','High',report_return,CHIPS_return]
    
    return reported_differences

def other_prio(x):
    if type(x) != str:
        return ['Other reason, could not determine','High']
    if any(True if y in x else False for y in ['shares','voting','%','ownership','right to appoint','significant influence']):
        return ['Nature of Control in text','High']
    if 'remove' in x or 'removal' in x:
        return ['Removal in text','High']
    return ['Other reason, could not determine','High']

def inc_dets_prio(x):
    if type(x) != str:
        return ['Incorrect details, could not determine','High']
    if any(True if y in x else False for y in ['shares','voting','%','ownership','right to appoint','significant influence']):
        return ['Nature of Control in text','High']
    if 'remove' in x or 'removal' in x:
        return ['Removal in text','High']
    return ['Incorrect details, could not determine','High']

def run_extract(psc_id,df):
#    print("Current ID : ",psc_id)
    ind = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    discrep_opt = df.loc[ind,'discrepancy_option_all']
    
    done_types = {
        'name':name_compare,'nationality':nationality_compare,'company name':company_name_compare,
        'company number':company_number_compare, 'date of birth':dob_compare
    }
    
    if discrep_opt in done_types.keys():
        op = done_types[discrep_opt](psc_id)
        return op
    elif ',' in discrep_opt:
        opts_list = discrep_opt.split(', ')
        
        high_markers = ['company name','company number','other reason','nature of control','date of birth','name']
        
        test_for_high = [True if x in high_markers else False for x in opts_list]
        
        if any(test_for_high):
            return ['Multi-High','High',None]
        else:
            return ['Multi-Low','Low',None]

# Bringing All Together

In [ ]:
# action smart survey action code 28a nov
# officer nationality dob smart survey
# data 

In [ ]:
%%time
path = 'PATH TO FILE'
df,df_off,df_act,df_removed = load_preprocess_data(path,survey_file = 'surveyMergedFinal.csv',
                                                   officer_file = 'NOC smart survey 6Dec dates.csv',
                                                action_file = 'smart survey action code 28a nov.csv')
print([df.shape,df_off.shape,df_act.shape])

df = run_extractions(df)
df.loc[df.discrepancy_option_all.isna(),'discrepancy_option_all'] = 'other reason'
inds_beginning = df.index

print("START SHAPE : ",df.shape[0])
df['priority_explained'] = None
df['priority'] = None
df['CHIPS_value'] = None

done_types = {'name':name_compare,'nationality':nationality_compare,'company name':company_name_compare,'company number':company_number_compare,'date of birth':dob_compare}

select_bool = [True if x in done_types.keys() or ',' in str(x) else False for x in df.discrepancy_option_all.values]

df_so_far = df.loc[select_bool]
df_so_far.columns = df_so_far.columns.str.replace(' ','')
  
explanations = []
priorities = []
CHIPS_values = []

for psc_id in df_so_far.PSC_DISCREPANCY_ID.values:
    ind = df_so_far[df_so_far.PSC_DISCREPANCY_ID == psc_id]
    
    op = run_extract(psc_id,df_so_far)
#    print("\nOP : ",op)

    explanations.append(op[0])
    priorities.append(op[1])
    CHIPS_values.append(op[2])
    
df_so_far['priority_explained'] = explanations
df_so_far['priority'] = priorities
df_so_far['CHIPS_value'] = CHIPS_values


df_noc = df.loc[df.discrepancy_option_all == 'nature of control']
print("No. NOCS : ",df_noc.shape[0])

chips_vals = []
discrep_info = []
for psc_id in df_noc.PSC_DISCREPANCY_ID.values:
    ind = df_noc.loc[df_noc.PSC_DISCREPANCY_ID == psc_id].index[0]
    op = noc_compare(psc_id)
    
    df_noc.loc[ind,'priority_explained'] = op[0]
    df_noc.loc[ind,'priority'] = op[1]
    
    if len(op) == 4:
        chips_vals.append(op[3])
        discrep_info.append(op[2])
    else:
        chips_vals.append(None)
        discrep_info.append(None)

df_noc.loc[:,'CHIPS_value'] = chips_vals
df_noc.loc[:,'discrepancy_information'] = discrep_info

# ===> Doing the remaining cats...
# PSC Remove
df_rem = df[df.discrepancy_option_all == 'psc remove']
df_rem['priority_explained'] = 'PSC Remove is High'
df_rem['Priority'] = 'High'

# Address
df_add = df[df.discrepancy_option_all == 'Address']
df_add['priority_explained'] = 'Address - Type unknown'
df_add['priority'] = 'Low'

# Place of Residence
df_por = df[df.discrepancy_option_all == 'place of residence']
df_por['priority_explained'] = 'Place of Residence is Material Low'
df_por['priority'] = 'Low'

# Country of Residence
df_cor = df[df.discrepancy_option_all == 'country of residence']
df_cor['priority_explained'] = 'Country of Residence is Non-Material'
df_cor['priority'] = 'None'

# PSC Missing
df_mis = df[df.discrepancy_option_all == 'psc missing']
df_mis['priority_explained'] = 'PSC Missing'
df_mis['priority'] = 'High'

# Legal form
df_lf = df[df.discrepancy_option_all == 'legal form']
df_lf['priority_explained'] = 'Legal form is Low'
df_lf['priority'] = 'Low'

# Incorporated In
df_ii = df[df.discrepancy_option_all == 'incorporated in']
df_ii['priority_explained'] = 'Incorporation Law'
df_ii['priority'] = 'Low'

# Correspondence Address
df_ca = df[df.discrepancy_option_all == 'correspondence address']
df_ca['priority_explained'] = 'Correspondence Address is Low'
df_ca['priority'] = 'Low'

# Place registered
df_pr = df[df.discrepancy_option_all == 'place registered']
df_pr['priority_explained'] = 'Place Registered is Low'
df_pr['priority'] = 'Low'

# Other parts

df_inc = df[(df.discrepancy_option_all == 'incorrect psc details') | (df.discrepancy_option_all == 'incorrect company details')]
op = df_inc.Contact_CLOB.apply(inc_dets_prio)

### TO FIX THIS GO THROUGH JUST ON ID AND ASSIGN VIA .LOC JUST TO GET IT DONE
exps = []
prios = []
for ind in df_inc.index:
    clob = df_inc.loc[ind,'Contact_CLOB']
    op = inc_dets_prio(clob)
    exps.append(op[0])
    prios.append(op[1])

df_inc['priority_explained'] = exps
df_inc['priority'] = prios

df_other = df[df.discrepancy_option_all == 'other reason']
op = df_other.Contact_CLOB.apply(other_prio).values

exps = []
prios = []
for ind in df_other.index:
    clob = df_other.loc[ind,'Contact_CLOB']
    op = other_prio(clob)
    exps.append(op[0])
    prios.append(op[1])
    
    
df_other['priority_explained'] = exps
df_other['priority'] = prios

# Merging All Together

frames = [df_so_far,df_noc,df_rem,df_add,df_por,df_cor,df_mis,df_lf,df_ii,df_ca,df_pr,df_inc,df_other]

df_so_far = pd.concat(frames)


# tagging material non-material
df_so_far.loc[df_so_far.priority.isna(),'priority'] = 'None'
df_so_far['Material_NonMaterial'] = df_so_far['priority'].apply(lambda x: 'Material' if str(x) != 'None' else 'Non-Material')
df_so_far['Single/Multi Discrepancy'] = df_so_far.discrepancy_option_all.apply(lambda x: 'Multi' if ',' in x else 'Single')


# column select
cols_to_keep = ['PSC_DISCREPANCY_ID','CORPORATE_BODY_ID','CREATED_DATE',
               'PSC_DISCREPANCY_TYPE_DESC','Contact_CLOB','information_provided','discrepancy_extract','discrepancy_option_all',
               'Single/Multi Discrepancy','PSC_NAME','relevant_information','discrepancy_information','CHIPS_value','priority_explained',
                'priority','Material_NonMaterial']

# 'DISCREPANCY_COMPANY_NUMBER','Organization',

df_keep = df_so_far[cols_to_keep]
df_keep = df_keep.sort_values(by='CREATED_DATE')

# Below is because values notbeing assigned correctly
df_keep.loc[df_keep.priority_explained == 'PSC Remove is High','priority'] = 'High'
df_keep.loc[df_keep.priority_explained == 'PSC Remove is High','Material_NonMaterial'] = 'Material'

#df_keep['DISCREPANCY_COMPANY_NUMBER'] = df_keep['DISCREPANCY_COMPANY_NUMBER'].apply(lambda x: x.replace("'",'') if type(x) == str else x)
inds_end = df_keep.index

print(df_keep.shape)
df_keep.sample(10)

In [ ]:
df_keep.priority_explained.value_counts()

In [ ]:
df_keep.shape

In [ ]:
df_keep[df_keep.priority_explained == 'Too many controls - NOC'].head(20)

In [ ]:
pd.read_csv("PATH TO FILE", encoding = 'iso-8859-1')

In [ ]:
#df_keep[df_keep.discrepancy_option_all == 'nationality']

In [ ]:
df_keep.to_csv("TO COMPARE.csv",index = False)